In [1]:
# This notebook is  used to evaluate perofrmance of our RAG ingestion and query pipeline.
# Method:
#   - Generated test data using a sample GitHub repository (offline process not in this notebook)
#     - https://docs.ragas.io/en/stable/getstarted/rag_testset_generation/
#   - Use our ingestion pipeline to parse the same reporisiroty and index in our vector store
#   - Use RAGAS on teadt dataset + our answers fromn our RAG and visualise the mterics.
import os
import sys
import pandas as pd

sys.path.insert(1, '/home/jovyan/work/code')
from opentelemetry import trace
from config import VectorDBConfig, EmbeddingConfig, ProcessingConfig, ChatConfig
from config_helper import ConfigHelper
from pipeline import DocumentPipeline
from localrag import LocalRAG
import nest_asyncio
import asyncio 
import logging
 
# Enable nested event loops
nest_asyncio.apply()

# We are using Aspire. Of course we will see the telemetry and logs in our dashboard!
# see config_helper.py for the not to tidy details.
tracer = trace.get_tracer(__name__)

config_helper = ConfigHelper(False)


In [2]:
# Ingest a GitHub repository and import into our vector store

# Initialize the pipeline
pipeline = DocumentPipeline(
    vector_db_config=config_helper.vector_db_config,
    embedding_config=config_helper.embedding_config
)

###Process git repository
repository="https://github.com/dotnet/docs-aspire"
with tracer.start_as_current_span(f"Starting ingesting {repository}"):
    pipeline.process_single_file("dotnet-docs-aspire.txt",repository)



/home/jovyan/work/code/pipeline.py:36: UserWarning: Api key is used with an insecure connection.
  self.qdrant = QdrantClient(url=vector_db_config.url, api_key=vector_db_config.api_key)


Total parts found: 439
Total files processed: 219
Sample file paths:
 1. README.md
 2. CODE_OF_CONDUCT.md
 3. SECURITY.md
Added 1 chunks for file .pre-commit-config.yaml
Added 22 chunks for file .repoman.yml
Added 72 chunks for file docs/index.yml
Added 73 chunks for file docs/toc.yml
Added 2 chunks for file docs/breadcrumb/toc.yml
Added 5 chunks for file docs/compatibility/toc.yml
Added 50 chunks for file docs/reference/aspire-faq.yml
Added 18 chunks for file docs/whats-new/index.yml
Added 6 chunks for file docs/whats-new/toc.yml
Added 7 chunks for file docs/zones/zone-pivot-groups.yml
Added 186 chunks for file .github/dependabot.yml
Added 3 chunks for file .github/ISSUE_TEMPLATE/01-general-issue.yml
Added 9 chunks for file .github/ISSUE_TEMPLATE/02-docs-request.yml
Added 5 chunks for file .github/ISSUE_TEMPLATE/03-customer-support.yml
Added 12 chunks for file .github/ISSUE_TEMPLATE/04-breaking-change.yml
Added 8 chunks for file .github/ISSUE_TEMPLATE/05-community-toolkit-docs-request

In [4]:
# Test our RAG Solution

def demonstrate_local_rag(rag):
    """Demonstrate how to use the LocalRAG class."""    
    # Example questions to test
    questions = [
        "If I use Docker Compose, why should I care about .Net Aspire?",
        "Is .Net Aspire an alternative to Kubernetes?"
    ]
    with tracer.start_as_current_span("Entering questions loop."):
        for question in questions:
            print(f"Question: {question}")
            # print("\nRelevant chunks:")
            # with tracer.start_as_current_span("rag get chunks"):
            #     chunks = rag.get_relevant_chunks(question, k=5)
            #     for i, chunk in enumerate(chunks, 1):
            #         print(chunk)
            #         break
#                    print(f"\nChunk {i}:")
#                    print(f"Source: {chunk.metadata.get('file_path', 'Unknown')}")
#                    print(f"Content: {chunk.page_content[:200]}...")
                    
            print("\nGenerated Answer:")
            with tracer.start_as_current_span("Retrieve answers."):
                answer = rag.retrieve_and_answer(question, k=6)
                print(answer)
                print("\n" + "="*80 + "\n")

rag = LocalRAG(
    vector_db_config=config_helper.vector_db_config,
    embedding_config=config_helper.embedding_config, 
    chat_config=config_helper.chat_config
)
with tracer.start_as_current_span("Starting demo"):
    demonstrate_local_rag(rag)

/home/jovyan/work/code/localrag.py:42: UserWarning: Api key is used with an insecure connection.
  self.qdrant = QdrantClient(url=vector_db_config.url, api_key=vector_db_config.api_key)


Question: If I use Docker Compose, why should I care about .Net Aspire?

Generated Answer:
If you're using Docker Compose for managing multiple projects or executables and want to run the front-end, backend, workers, and database as a single application, you might not need to know anything beyond what you already know about .NET Aspire. However, there are several reasons why you might still consider using it:

1. **Simpllicity**: .NET Aspire is designed to simplify the development process by allowing you to define applications as a series of interconnected services without worrying about containerization and orchestration.
2. **Easy setup**: Once set up, .NET Aspire can handle scaling and deployment out of the box, eliminating the need for manual Docker management.
3. **Orchestrator role**: .NET Aspire acts as an orchestrator, managing the lifecycle of applications, including containers, services, and database connections.
4. **Real-world application**: Many real-world applications use